In [14]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.sparse import hstack
import pickle
from tqdm import tqdm
import pickle

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [16]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [17]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout, Input, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
df=pd.read_csv("task-1/data/cleaned_data.csv")
y_true=df['is_duplicate']

In [22]:
# transformer=pickle.load(open("task-1/data/tfidf/tfidf_transformer.pkl", "rb"))

In [23]:
question1=df["question1"][:5000]
question2=df["question2"][:5000]

In [24]:
all_questions = list(question1) + list(question2)
len(all_questions)

10000

In [25]:
vectorizer = TfidfVectorizer()
# analyze = vectorizer.build_analyzer()
transformer=vectorizer.fit(all_questions)

In [26]:
question1_vectors=transformer.transform(question1)
question2_vectors=transformer.transform(question2)

In [27]:
question1_vectors = question1_vectors.todense()
question2_vectors = question2_vectors.todense()

In [28]:
question1_vectors[5].shape

(1, 10183)

In [29]:
x=np.hstack((question1_vectors[:],question2_vectors[:]))
x.shape

(5000, 20366)

In [30]:
max(x[1])

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y_true[:5000],test_size=0.2,random_state=40)

In [32]:
model=Sequential()
model.add(GRU(128,activation="relu",input_shape=(20366,1,)))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [33]:
callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            min_delta=1e-5,
            patience=10,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath="task-1/saved_models/tfidf/gru",
            monitor='val_loss', 
            mode='min', 
            save_best_only=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2,
            patience=4, 
            min_lr=0.001)
    
]

optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy()

In [34]:
model.compile(loss = loss, optimizer = optimizer, metrics = acc)

In [ ]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data= (x_test, y_test), callbacks=callbacks, verbose=1)

Epoch 1/10
125/125 [==============================] - ETA: 0s - loss: 0.6926 - binary_accuracy: 0.6173WARNING:tensorflow:Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,binary_accuracy,val_loss,val_binary_accuracy

Epoch 1: val_loss improved from inf to 0.69204, saving model to task-1/saved_models/tfidf/gru
INFO:tensorflow:Assets written to: task-1/saved_models/tfidf/gru/assets
125/125 [==============================] - 901s 7s/step - loss: 0.6926 - binary_accuracy: 0.6173 - val_loss: 0.6920 - val_binary_accuracy: 0.6190 - lr: 1.0000e-05
Epoch 2/10
110/125 [=========================>....] - ETA: 1:42 - loss: 0.6915 - binary_accuracy: 0.6148